# EKE

This notebook takes raw, daily SSH data from the `OCCIPUT` simulations and estimates `EKE(x,y,t)` assuming surface geostrophic velocities. It then resamples down to monthly data and saves the output locally for further analysis.

**A few notes on this process:**
 * The tripolar grid in NEMO doesn't work so well with `xarray`. For that reason, these calculations are only really valid southwards of 23°N.
 * xarray handles gradients poorly. Since I am not using dask, for now, and numpy handles gradients a bit better, I have pulled out the SSH data as a numpy array, calculated SSH gradients from this, and pushed it back to a new dataarray, with proper latitude and longitude dimensions (south of 23°N)
 * The gradient calculation omits the variation of the gridsize with latitude. The EKE calculation also assumes constant f. This version is just approximate, to see if it is worth doing properly.
 * It is slow, and memory-intensive... for now I have limited the calculation just to the Southern Ocean region.

**AH - 4 April 2019**

In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import xarray as xr
import numpy as np
import IPython.display
import pandas as pd
import cmocean as cm
import cartopy.crs as ccrs
import cartopy.feature as cft
#from dask.distributed import Client, progress

import logging
logging.captureWarnings(True)
logging.getLogger('py.warnings').setLevel(logging.ERROR)

In [2]:
HardDisk = '/Volumes/Untitled/'
EnsembleDir = 'SSH_ENSEMBLE_all/'

In [3]:
# This is an approximate scaling factor to get EKE from grad H.
# It means EKE is in something like cm^2/s^2
# Need to improve in future versions.
ekefac = 9.8**2/(1e-4)**2/(0.25*1.e5)**2*1e4

In [4]:
%%time
filename = HardDisk+EnsembleDir+'ORCA025.L75-OCCITENS.001-S/1d/ORCA025.L75-OCCITENS.001_y1995.1d_SSH.nc'
ssh = xr.open_dataset(filename).ssh.drop('time_centered').isel(y=slice(130,310))
lon = ssh.nav_lon.isel(y=0).values.copy()
lon[:430] = lon[:430]-360
lat = ssh.nav_lat.isel(x=387).values
print('Figured out lat and lon')

for ii in range(44,51):
    i00 = '%03d' % ii
    print('Now doing '+i00)

    Member = 'ORCA025.L75-OCCITENS.'+i00
    file = 'ORCA025.L75-OCCITENS.'+i00+'_y????.1d_SSH.nc'
    filename = HardDisk+EnsembleDir+Member+'-S/1d/'+file
    ## Load SSH  - Southern Ocean only
    ssh = xr.open_mfdataset(filename).ssh.drop('time_centered').isel(y=slice(130,310))
    print('Opened '+filename)
        
    sla = ssh - ssh.mean('time_counter')
    print('Got SLA as a dataarray')
    
    grad_sla = np. gradient(sla.values,axis=(1,2))
    print('Calculated the gradient of SLA')
    
    eke = ekefac*(grad_sla[0]**2 + grad_sla[1]**2)
    print('Calculated EKE')

    ekeda  = xr.DataArray(eke, coords=[('time', ssh.time_counter.values), 
                                            ('lat', lat),
                                            ('lon', lon)], name='EKE',)
    print('Got EKE as a dataarray')
    
    eke_monthly = ekeda.resample(time='M').mean('time')
    print('Resampled EKE onto monthly')
    
    eke_monthly.to_netcdf('Data/'+Member+'_eke.nc')

Figured out lat and lon
Now doing 044
Opened /Volumes/Untitled/SSH_ENSEMBLE_all/ORCA025.L75-OCCITENS.044-S/1d/ORCA025.L75-OCCITENS.044_y????.1d_SSH.nc
Got SLA as a dataarray
Calculated the gradient of SLA
Calculated EKE
Got EKE as a dataarray
Resampled EKE onto monthly
Now doing 045
Opened /Volumes/Untitled/SSH_ENSEMBLE_all/ORCA025.L75-OCCITENS.045-S/1d/ORCA025.L75-OCCITENS.045_y????.1d_SSH.nc
Got SLA as a dataarray
Calculated the gradient of SLA
Calculated EKE
Got EKE as a dataarray
Resampled EKE onto monthly
Now doing 046
Opened /Volumes/Untitled/SSH_ENSEMBLE_all/ORCA025.L75-OCCITENS.046-S/1d/ORCA025.L75-OCCITENS.046_y????.1d_SSH.nc
Got SLA as a dataarray
Calculated the gradient of SLA
Calculated EKE
Got EKE as a dataarray
Resampled EKE onto monthly
Now doing 047
Opened /Volumes/Untitled/SSH_ENSEMBLE_all/ORCA025.L75-OCCITENS.047-S/1d/ORCA025.L75-OCCITENS.047_y????.1d_SSH.nc
Got SLA as a dataarray
Calculated the gradient of SLA
Calculated EKE
Got EKE as a dataarray
Resampled EKE onto 

In [ ]:
plt.figure(figsize=(12,5))
ax = plt.subplot(1,1,1,projection=ccrs.Robinson(central_longitude=0))
ax.coastlines(resolution='50m')
#ax.add_feature(land_50m)
#plt.pcolormesh(lon,lat,ekeda,vmin=-1.5,vmax=1.5,cmap=cm.cm.balance,transform=ccrs.PlateCarree())
eke_monthly.mean('time').plot(vmax=1000,cmap=cm.cm.thermal,transform=ccrs.PlateCarree())#vmin=0,vmax=2000,
#plt.colorbar(shrink=0.6,extend='both')

In [ ]:
eke_monthly.sel(lon=slice(-210,-72)).sel(lat=slice(-62,-48)).mean('lon').mean('lat').plot()